# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [1]:
# Import your libraries:

import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split     
from sklearn.model_selection import cross_val_score

# Challenge 1 - Explore The Data

This lesson will explore the creation of a machine learning pipeline from beggining to end. We will save our model and use the model to make predictions on data outside of our training sample. Let's start by loading the dataset which can be obtained from [Kaggle](https://www.kaggle.com/uciml/mushroom-classification) and [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/mushroom).

In [2]:
# Loading the data

mushrooms = pd.read_csv('../mushrooms.csv')

This dataset contains information about different types of mushrooms. Our response variable indicates whether a mushroom is poisonous. 

####  We will create a model to predict whether a mushroom is poisonous (the `class` column) using the information in all other columns.

Let's print the `head()` of this dataset to see what columns we have in our data.

In [3]:
mushrooms.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


#### It looks like the columns in this dataset are coded. 

Let's examine the column types using `dtypes` to confirm this. 

In [4]:
mushrooms.dtypes


class                       object
cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
dtype: object

In this dataset, each column (feature) of the mushroom is represented by a single-character code. It would be best if we can obtain a dictionary of these codes so that we know what possible codes each column has and what each code represents. This dictionary can be obtained from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/mushroom). In the table below, we print the code dictionary for you to reference.

| Features | Codes |
|---|---|
| classes | edible=e, poisonous=p |
| cap-shape | bell=b,conical=c,convex=x,flat=f,knobbed=k,sunken=s |
| cap-surface | fibrous=f,grooves=g,scaly=y,smooth=s |
| cap-color | brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y |
| bruises | bruises=t,no=f |
| odor | almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s |
| gill-attachment | attached=a,descending=d,free=f,notched=n |
| gill-spacing | close=c,crowded=w,distant=d |
| gill-size | broad=b,narrow=n |
| gill-color | black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y |
| stalk-shape | enlarging=e,tapering=t |
| stalk-root | bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=? |
| stalk-surface-above-ring | fibrous=f,scaly=y,silky=k,smooth=s |
| stalk-surface-below-ring | fibrous=f,scaly=y,silky=k,smooth=s |
| stalk-color-above-ring | brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y |
| stalk-color-below-ring | brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y |
| veil-type | partial=p,universal=u |
| veil-color | brown=n,orange=o,white=w,yellow=y |
| ring-number | none=n,one=o,two=t |
| ring-type | cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z |
| spore-print-color | black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y |
| population | abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y |
| habitat | grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d


**The columns in the mushrooms dataset seem to be correlated**. We suspect this because many columns seem to have dependencies. For example, if `ring-number` is `n` (none), `ring-type` must be `n` (none) too. Only when `ring-number` is `o` (one) or `t` (two) `ring-type` can be values other than `n`. Other columns may have inter-dependencies too but we can't tell right now. 

Why are we concerned about the variable correlations (also called *multicollinearity*)? It's because if they are strongly correlated, we shouldn't use the linear regression algorithm as the machine learning model. The prediction of the linear regression model is known to be unstable and inaccurate if the variables are strongly correlated. You can read more about why [here](https://en.wikipedia.org/wiki/Multicollinearity#Consequences_of_multicollinearity).

So far it's just our suspicion that the variables are strongly correlated. We need to verify that. Since all the columns are categorical, we cannot use a correlation matrix to examine the degrees of correlations. There are several ways to test whether categorical variables are correlated. The first way is to convert the categorical values to ordinal, then calcualte the correlation matrix with the numerical values. The second way is to use a [Chi-Square test of independence](https://onlinecourses.science.psu.edu/stat500/node/56/) to find out whether there is correlation between each pair of variables. We will use the second way in this lab. So read the explanation of Chi-Square test of independence to understand what it does.

We will begin with the first 2 variables (`cap-shape` and `cap-surface`). Our **null hypothesis of the Chi-Square test of independence is that the two variables are independent** and the **alternative hypothesis is that they are not independent**. To perform Chi-Squre test of independence, we first create a contingency table of those two features using Pandas' [`crosstab`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html) function, then use Scipy's [`chi2_contingency`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html) to test the variable independence from the contingency table.

#### In the following cell, create a contingency table of `cap-surface` and `cap-shape`. Assign this table to the variable `ct`.

In [5]:
ct = pd.crosstab(mushrooms['cap-surface'], mushrooms['cap-shape'])
ct


cap-shape,b,c,f,k,s,x
cap-surface,,,,,,
f,52,0,1016,60,32,1160
g,1,1,1,1,0,0
s,244,0,820,418,0,1074
y,155,3,1315,349,0,1422


Let's import the chi quare test:

In [6]:
from scipy.stats import chi2_contingency

In the following cell perform the chi square test on the contingency table using the function you just imported. This function wil return a tuple with 4 values. The second value in the tuple is the p-value for our test. Print the p-value and interpret the result - is the null hypothesis rejected? 

In [7]:
chi_cont=chi2_contingency(ct)
chi_cont


(1011.4930034066986,
 4.635777687474967e-206,
 15,
 array([[1.29079271e+02, 1.14229444e+00, 9.00128016e+02, 2.36454948e+02,
         9.13835549e+00, 1.04405711e+03],
        [2.22550468e-01, 1.96947317e-03, 1.55194485e+00, 4.07680945e-01,
         1.57557853e-02, 1.80009847e+00],
        [1.42209749e+02, 1.25849335e+00, 9.91692762e+02, 2.60508124e+02,
         1.00679468e+01, 1.15026292e+03],
        [1.80488429e+02, 1.59724274e+00, 1.25862728e+03, 3.30629247e+02,
         1.27779419e+01, 1.45987986e+03]]))

In [8]:
# EL pvalor es casi 0. Rechazamos la hipótesis nula. SOn dependientes.

## Now let's create a p-value matrix for all variables.

In the cell below, create a 2-dimensional matrix of all pairwise tests for each two features. Print the matrix and interpret the results.

Below is an example of what your matrix should look like:
![corr df](../corr_df.png)

*Hint: Use loop inside loop to perform pairwise tests for each two features.*

In [9]:
columnas = mushrooms.columns
temp = {}
for i in columnas:
    temp_lista = []
    for j in columnas:
        ct_temp = pd.crosstab(mushrooms[i],mushrooms[j])
        temp_lista.append(chi2_contingency(ct_temp)[1])
    temp[i] = temp_lista
matrix_pvalue = pd.DataFrame(temp, index = columnas)
matrix_pvalue



,bruises,cap-color,cap-shape,cap-surface,class,gill-attachment,gill-color,gill-size,gill-spacing,habitat,...,ring-type,spore-print-color,stalk-color-above-ring,stalk-color-below-ring,stalk-root,stalk-shape,stalk-surface-above-ring,stalk-surface-below-ring,veil-color,veil-type
class,0.000000e+00,6.055815e-78,1.196457e-103,5.518427e-68,0.000000e+00,5.501707e-31,0.000000e+00,0.000000e+00,5.022978e-216,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.702048e-290,4.604746e-20,0.000000e+00,0.000000e+00,3.320973e-41,1.0
cap-shape,1.811119e-112,1.249975e-220,0.000000e+00,4.635778e-206,1.196457e-103,3.135646e-40,0.000000e+00,5.169179e-209,1.879772e-10,0.000000e+00,...,1.793004e-286,0.000000e+00,0.000000e+00,2.928263e-221,0.000000e+00,2.372578e-163,2.363147e-123,1.911102e-56,1.661712e-252,1.0
cap-surface,1.359844e-31,1.237998e-243,4.635778e-206,0.000000e+00,5.518427e-68,5.758186e-78,0.000000e+00,3.449797e-135,3.831131e-201,0.000000e+00,...,3.975737e-256,1.076398e-263,0.000000e+00,0.000000e+00,0.000000e+00,6.542808e-11,2.663960e-103,9.621957e-145,3.845602e-89,1.0
cap-color,2.347626e-77,0.000000e+00,1.249975e-220,1.237998e-243,6.055815e-78,7.290466e-102,0.000000e+00,0.000000e+00,3.859462e-274,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.121524e-100,1.0
bruises,0.000000e+00,2.347626e-77,1.811119e-112,1.359844e-31,0.000000e+00,8.033466e-35,0.000000e+00,5.662661e-243,4.139756e-160,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.134871e-19,0.000000e+00,0.000000e+00,2.125442e-31,1.0
odor,0.000000e+00,0.000000e+00,0.000000e+00,5.624295e-278,0.000000e+00,4.518962e-115,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.514334e-43,1.0
gill-attachment,8.033466e-35,7.290466e-102,3.135646e-40,5.758186e-78,5.501707e-31,0.000000e+00,0.000000e+00,1.891824e-22,2.173300e-10,0.000000e+00,...,3.125969e-109,0.000000e+00,0.000000e+00,0.000000e+00,8.015774e-85,6.936860e-63,1.134492e-15,3.357803e-27,0.000000e+00,1.0
gill-spacing,4.139756e-160,3.859462e-274,1.879772e-10,3.831131e-201,5.022978e-216,2.173300e-10,8.717651e-178,2.206874e-22,0.000000e+00,0.000000e+00,...,7.130888e-147,9.687671e-162,6.100406e-277,5.622180e-242,0.000000e+00,3.843922e-13,0.000000e+00,6.959008e-303,4.867412e-17,1.0
gill-size,5.662661e-243,0.000000e+00,5.169179e-209,3.449797e-135,0.000000e+00,1.891824e-22,0.000000e+00,0.000000e+00,2.206874e-22,0.000000e+00,...,0.000000e+00,0.000000e+00,1.043350e-229,4.664424e-206,0.000000e+00,3.923369e-83,1.176480e-50,7.224177e-40,1.006477e-22,1.0
gill-color,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.717651e-178,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0


In [10]:
# Your comment here

We verified the variables are highly correlated from the pairwise Chi-Square test of independence.

#### The next step in model generation is to ensure there is no missing data and handle any missing data if they exist.

In the next cell, check to see if there is any missing data in each column of the dataset

In [11]:
matrix_pvalue.info()


<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, class to habitat
Data columns (total 23 columns):
bruises                     23 non-null float64
cap-color                   23 non-null float64
cap-shape                   23 non-null float64
cap-surface                 23 non-null float64
class                       23 non-null float64
gill-attachment             23 non-null float64
gill-color                  23 non-null float64
gill-size                   23 non-null float64
gill-spacing                23 non-null float64
habitat                     23 non-null float64
odor                        23 non-null float64
population                  23 non-null float64
ring-number                 23 non-null float64
ring-type                   23 non-null float64
spore-print-color           23 non-null float64
stalk-color-above-ring      23 non-null float64
stalk-color-below-ring      23 non-null float64
stalk-root                  23 non-null float64
stalk-shape                 2

#### Since there is no work to be done to clean up missing data, the next step is to create dummy variables. 

Most machine learning algorithms cannot work with non-numeric data, so we will need to transform our data. Use the [`get_dummies` function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) to transform the data. Make sure to remove one dummy column per variable using the `drop_first=True` option.

In [12]:
mushrooms_dummy = pd.get_dummies(mushrooms, drop_first = True)
mushrooms_dummy.head()


,class_p,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


#### Our final data exploration task is to prepare the data for modeling by splitting it to predictor, response, train and test. 

We will do this using the [`train_test_split` function from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). In the cell below, split the data to `X_train`, `X_test`, `y_train`, and `y_test` using this function. Select 80% of the data for the training sample and the rest for the test sample.

In [13]:
from sklearn.model_selection import train_test_split

X = mushrooms_dummy.drop(axis=1, labels=["class_p"])
y = mushrooms_dummy['class_p']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
clf = svm.SVR(gamma = 'auto').fit(X_train, y_train)
clf.score(X_test, y_test)

0.9768868448159593

# Challenge 2 - Creating and Saving Our Model

Determining whether a mushroom is poisonous is a classification task. There are multiple classification models we can choose from.
However, since we have determined that there are many columns that are not indepdendent, this limits our choice of model. One model we will not consider is logistic regression. Two potential choices for this modeling task are [random forest](https://en.wikipedia.org/wiki/Random_forest) and [SVM](https://en.wikipedia.org/wiki/Support_vector_machine).

Let's start with Random Forest. We think of random forest as a voting algorithm. We generate many decision trees by sampling both rows and columns in our dataset. Each one of these trees produces a decision. We let all the trees "vote" and the aggregate decision that they produce gives us the final decision for our algorithm (in this case, they will vote whether each mushroom is poisonous or edible). To learn more about random forests, click [here](https://onlinecourses.science.psu.edu/stat857/node/179/).

In the cell below, we will import and initialize a random forest from scikit-learn ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). Assign the initialized model to `mushroom_rf`. For now, we will just use the default settings for the random forest classifier, so there is no need to pass any arguments to the function.

In [14]:
from sklearn.ensemble import RandomForestClassifier

mushroom_rf = RandomForestClassifier()


In the cell below, fit the model to the training data.

In [15]:
mushroom_rf.fit(X, y) 
#print(mushroom_rf.feature_importances_)

/home/eduardo/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Next, let's evaluate the model. One of the most straightforward ways to evaluate a classification model is using a confusion matrix. 

The confusion matrix shows us the true positives, false positives, false negatives and true negatives in the data. Our goal is to maximize the true positives and true negatives (the observations that are correctly classified) and minimize the false positives and false negatives.

In the cell below, we'll start by generating predictions for the test data using the `predict` function. 

In [16]:
y_pred = mushroom_rf.predict(X_test)


Now we'll import the `confusion_matrix` function ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)) and compute the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) by comparing the observed data (`y_test`) and the predicted data that you found in the cell above.

In [17]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[795,   0],
       [  0, 830]])

# Bonus Challenge 1 - Use a Different ML Algorithm to Evaluate Model

Repeat the steps here to predict and evaluate the model but instead use gradient boosted classification. Your end result should be a confusion matrix comparing the predicted and observed y values for the test sample. You can read more about gradient boosting [here](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/).

To calculate the gradient boosted classfication, use the [`GradientBoostingClassifier` of Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

gr_boosted = GradientBoostingClassifier().fit(X_train, y_train)
prediction_gr = gr_boosted.predict(X_test)
print(prediction_gr)


[1 0 1 ... 1 1 0]


In [28]:
confusion_matrix(y_test, prediction_gr)

array([[795,   0],
       [  0, 830]])

# Challenge 3 - Producing Individual Predictions and Saving The Model

One of the most important goals of machine learning models is to act as something like a prediction black box. We would like to pass an observation to the model and get back a prediction as an output. Let's do this in the next cells using the `predict` function. What we want to do is to pick a random mushroom and generate a prediction that will tell us whether it is poisonous.

#### In the next cell, create a function called `get_random_data` that accepts a dataset and returns a random row from the dataset.

In [29]:
from random import seed
from random import randint

def get_random_data(dataset):
    """
    Returns a random row of a dataset
    
    Args:
        dataset [dataframe]: a Pandas dataframe containing a dataset
    
    Returns:
        A random row in the dataset
    """
    # Your code here
    return dataset.sample()

#### Call `get_random_data` to obtain a random row from the test dataset. Assign the returned row to `random_mushroom` and print.

In [37]:
random_mushroom = get_random_data(mushrooms_dummy)


#### In the cell below, use the `predict` function to generate a prediction for this random mushroom. Is the random mushroom poisonous? Compare this to the true y value.

In [41]:
testing = random_mushroom.drop(axis=1, labels=["class_p"])
poison_test = gr_boosted.predict_proba(testing)
poison = gr_boosted.predict(testing)

print(poison_test)
print(poison)


[[9.99227653e-01 7.72346647e-04]]
[0]


In [42]:
# La seta es venenosa

#### Our final step is to save our model. 

Do this in the cell below using [pickling](https://docs.python.org/3/library/pickle.html). Import the pickle library and save the `mushroom_rf` model as a pickle file using `pickle.dump()`. Name your file `mushrooms.sav`

In [43]:
import pickle

pickle_out = open('mushrooms.sav','wb')
pickle.dump(mushroom_rf, pickle_out)
pickle_out.close()


## Bonus Challenge 2 - Heatmap to Visualize Data Correlation

Practice generating a heatmap to visualize the pairwise column correlations. You can do one of the following two things (or both if you like):

* Create a heatmap using the p-value matrix for the pairwise Chi-Square test of independence. 

* Use Scikit-Learn's `LabelEncoder` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)) to transform the raw categorical variables to ordinal. Then calculate the correlation matrix using the ordinal values.

If you're scratching your hairs off, refer to [this example for the first way](https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9) and [this example for the second way](https://www.kaggle.com/haimfeld87/analysis-and-classification-of-mushrooms). 

In [24]:
# Your code here